In [1]:
include("../basis.jl")
include("../identity.jl")
include("../basic_gates.jl")
using Compat.Test
using BenchmarkTools

In [2]:
### pretty interface
for GATE in [:xgate, :ygate, :zgate, :ndiaggate, :diaggate, :controlled_U1, :cnotgate, :projector, :toffoligate, :swapgate]
    @eval $GATE(num_bit::Int, params...) = $GATE(basis(num_bit), params...)
end

In [3]:
function xgate(basis::Vector{DInt}, bits::Ints)
    norder = flip(basis, bits)
    PermuteMultiply(norder+1, ones(length(basis)))
end

function ygate(basis::Vector{DInt}, bits::Ints)
    norder = flip(basis, bits)
    #vals = [-im*(-1)^reduce(+,takebit(b, bits)) for b in basis]
    #vals = mapreduce(bit->map(x->x==0?-im:im, takebit(basis, bit)), .*, bits)
    vals = mapreduce(bit->im.*(2.*takebit(basis, bit).-1), .*, bits)
    PermuteMultiply(norder+1, vals)
end

function zgate(basis::Vector{DInt}, bits::Ints)
    #vals = [(-1)^reduce(+,takebit(b, bits)) for b in basis]
    vals = mapreduce(bit->1.-2.*takebit(basis, bit), .*, bits)
    PermuteMultiply(basis+1, vals)
end

zgate (generic function with 2 methods)

In [4]:
############################ TODO ################################
# arbitrary off-diagonal single qubit gate
# e.g. X, Y
function ndiaggate(basis::Vector{DInt}, gate::PermuteMultiply, bits::Ints)
    norder = flip(basis, bits)
    vals = mapreduce(bit->exp.(im*phi*(2.*takebit(basis, bit).-1)), .*, bits)
    PermuteMultiply(norder+1, vals)
end

# arbitrary diagonal single qubit gate
# e.g. Z, Rz(θ)
function diaggate(basis::Vector{DInt}, gate::Diagonal, bits::Ints)
    vals = mapreduce(bit->exp.(im*phi*(2.*takebit(basis, bit).-1)), .*, bits)
    PermuteMultiply(basis+1, vals) # or Diagonal(vals) ?
end

#TODO?
# arbituary dense single qubit gate: SparseMatrixCSC
# e.g. Rx(θ), Ry(θ), Rot(θ1,θ2,θ3)
#function densegate(bits::Ints, basis::Vector{DInt})
#end

# arbituary control PermuteMultiply gate: SparseMatrixCSC
# swap gate
function swapgate(basis::Vector{DInt}, b1::Int, b2::Int)
end

swapgate (generic function with 2 methods)

In [6]:
⊗ = kron
function superkron(num_bit::Int, gates::Vector{T}, locs::Vector{Int}) where T<:AbstractMatrix
    locs = num_bit - locs + 1
    order = sortperm(locs)
    _wrap_identity(gates[order], diff(vcat([0], locs[order], [num_bit+1])) - 1)
end

# kron, and wrap matrices with identities.
function _wrap_identity(data_list::Vector{T}, num_bit_list::Vector{Int}) where T<:AbstractMatrix
    length(num_bit_list) == length(data_list) + 1 || throw(ArgumentError())
    reduce((x,y)->x ⊗ y[1] ⊗ II(1<<y[2]), II(1 << num_bit_list[1]), zip(data_list, num_bit_list[2:end]))
end

_wrap_identity (generic function with 1 method)

In [7]:
@assert superkron(4, [PAULI_X, PAULI_Y], [3,2]) == II(2) ⊗ PAULI_X ⊗ PAULI_Y ⊗ II(2)

In [8]:
# a function projecting space into target space.
function projector(basis::Vector{DInt}, cbits::Int)
    # Assigned: LW
end

projector (generic function with 2 methods)

In [9]:
#@test projector(basis(6), [2, -3, -5]) == superkron(6, [P0, P1, P1], [2, 3, 5])

In [10]:
general_controlled_gates(num_bit::Int, projectors::Vector{Tp}, cbits::Vector{Int}, gates::Vector{Tg}, locs::Vector{Int}) where {Tg<:AbstractMatrix, Tp<:AbstractMatrix} = II(1<<num_bit) - superkron(num_bit, projectors, cbits) + superkron(num_bit, vcat(projectors, gates), vcat(cbits, locs))

general_controlled_gates (generic function with 1 method)

In [32]:
CNOT = PermuteMultiply(Complex128[1 0 0 0; 0 1 0 0; 0 0 0 1; 0 0 1 0])
TOFFOLI = kron(P0, II(4)) + kron(P1, CNOT)
@test general_controlled_gates(2, [P1], [2], [PAULI_X], [1]) == CNOT

Test Passed

In [33]:
@test zgate(4, [1,2,3]) == superkron(4, [PAULI_Z, PAULI_Z, PAULI_Z], [1,2,3])

Error During Test
  Test threw an exception of type MethodError
  Expression: zgate(4, [1, 2, 3]) == superkron(4, [PAULI_Z, PAULI_Z, PAULI_Z], [1, 2, 3])
  MethodError: no method matching zgate(::UnitRange{Int64}, ::Array{Int64,1})
  Closest candidates are:
    zgate(::Int64, ::Any...) at In[2]:3
    zgate(::Array{Int64,1}, ::Union{Array{Int64,1}, Int64, UnitRange{Int64}}) at In[3]:16
  Stacktrace:
   [1] zgate(::Int64, ::Array{Int64,1}) at ./In[2]:3
   [2] include_string(::String, ::String) at ./loading.jl:522
   [3] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/leiwang/.julia/v0.6/IJulia/src/execute_request.jl:158
   [4] (::Compat.#inner#18{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/leiwang/.julia/v0.6/Compat/src/Compat.jl:403
   [5] eventloop(::ZMQ.Socket) at /Users/leiwang/.julia/v0.6/IJulia/src/eventloop.jl:8
   [6] (::IJulia.##14#17)() at ./task.jl:335


LoadError: [91mThere was an error during testing[39m

In [82]:
# Assigned: LW
# control gate for single qubit gate
function controlled_U1(basis::UnitRange{DInt}, gate::PermuteMultiply{T}, b1::Int, b2::Int) where {T}
    dim = size(basis,1)
    vals = ones(T, dim)
    order = collect(1:dim)
    for b in basis
        if testbit(b, b1)
            @inbounds vals[b+1] = gate.vals[gate.perm[1+takebit(b, b2)]]
            @inbounds order[b+1] = (gate.perm[1] == 1) ? b+1: flip(b, b2)+1
        end
    end
    PermuteMultiply(order, vals)
end

function controlled_U1(basis::UnitRange{DInt}, gate::SparseMatrixCSC, b1::Int, b2::Int)
    general_controlled_gates(2, [P1], [b1], [gate], [b2])
end

function controlled_U1(basis::UnitRange{DInt}, gate::Diagonal{T}, b1::Int, b2::Int) where {T}
    #Diagonal([takebit(b, b1)==1 ? gate.diag[1+takebit(b, b2)]: 1 for b in basis])
    #a = (gate.diag[1]+gate.diag[2])/2.
    #b = (gate.diag[1]-gate.diag[2])/2.
    #a*II(1<<num_bit) + b*czgate(num_bit, b1, b2)
    dim = size(basis,1)
    vals = ones(T, dim)
    for b in basis
        if testbit(b, b1)
            @inbounds vals[b+1] = gate.diag[1+takebit(b, b2)]
        end
    end
    Diagonal(vals)
end
function controlled_U1(basis::UnitRange{DInt}, gate::StridedMatrix, b1::Int, b2::Int)
    general_controlled_gates(2, [P1], [b1], [gate], [b2])
end

controlled_U1 (generic function with 6 methods)

In [64]:
@test controlled_U1(2, PAULI_Z, 1, 2) == czgate(basis(2), 1, 2) 

Test Passed

In [81]:
#display(@benchmark controlled_U1(basis(16), $PAULI_Z, 3, 7))
display(@benchmark general_controlled_gates(16, [P1], [3], [$PAULI_Z], [7]))

BenchmarkTools.Trial: 
  memory estimate:  9.77 MiB
  allocs estimate:  116
  --------------
  minimum time:     2.767 ms (0.00% GC)
  median time:      5.092 ms (0.00% GC)
  mean time:        5.103 ms (12.73% GC)
  maximum time:     8.747 ms (18.04% GC)
  --------------
  samples:          978
  evals/sample:     1

In [17]:
@test controlled_U1(basis(4), PAULI_Z, 3, 1) == czgate(basis(4), 3, 1)

Error During Test
  Test threw an exception of type UndefVarError
  Expression: controlled_U1(basis(4), PAULI_Z, 3, 1) == czgate(basis(4), 3, 1)
  UndefVarError: czgate not defined
  Stacktrace:
   [1] include_string(::String, ::String) at ./loading.jl:522
   [2] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/leiwang/.julia/v0.6/IJulia/src/execute_request.jl:158
   [3] (::Compat.#inner#18{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/leiwang/.julia/v0.6/Compat/src/Compat.jl:403
   [4] eventloop(::ZMQ.Socket) at /Users/leiwang/.julia/v0.6/IJulia/src/eventloop.jl:8
   [5] (::IJulia.##14#17)() at ./task.jl:335


LoadError: [91mThere was an error during testing[39m

In [18]:
display(@benchmark controlled_U1(basis(10), PAULI_Z, 3, 7))

BenchmarkTools.Trial: 
  memory estimate:  72.34 KiB
  allocs estimate:  2565
  --------------
  minimum time:     76.347 μs (0.00% GC)
  median time:      81.556 μs (0.00% GC)
  mean time:        91.735 μs (5.67% GC)
  maximum time:     2.486 ms (93.34% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [37]:
# CNOT/CZ may be further accelerated.
function czgate(num_bit::Int, b1::Int, b2::Int)
    Diagonal(map(i->1-2*(takebit(i, b1) & takebit(i, b2)), basis(num_bit)))
end

czgate (generic function with 2 methods)

In [38]:
display(@benchmark czgate(16, 3, 7))

BenchmarkTools.Trial: 
  memory estimate:  512.09 KiB
  allocs estimate:  3
  --------------
  minimum time:     38.427 μs (0.00% GC)
  median time:      237.393 μs (0.00% GC)
  mean time:        218.233 μs (13.83% GC)
  maximum time:     1.997 ms (75.07% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [21]:
@test czgate(basis(2), 1, 2) == [1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 -1]
@test general_controlled_gates(16, [P1], [7], [PAULI_Z], [3]) == czgate(basis(16), 7, 3)

Test Passed

In [22]:
############# CNOT Gate as an example ##################
function cnotgate(basis::UnitRange{DInt}, b1::Int, b2::Int)
    @views order = xor.(basis, (basis .& bmask(b1)) .<< (b2-b1))
    PermuteMultiply(order+1, ones(Int, length(basis)))
end

cnotgate (generic function with 2 methods)

In [23]:
display(@benchmark cnotgate(16, 3, 7))

BenchmarkTools.Trial: 
  memory estimate:  1.50 MiB
  allocs estimate:  8
  --------------
  minimum time:     109.791 μs (0.00% GC)
  median time:      730.186 μs (0.00% GC)
  mean time:        634.986 μs (15.11% GC)
  maximum time:     2.615 ms (56.79% GC)
  --------------
  samples:          7842
  evals/sample:     1

In [24]:
@test cnotgate(basis(2), 2, 1) == [1 0 0 0; 0 1 0 0; 0 0 0 1; 0 0 1 0]
@test general_controlled_gates(16, [P1], [7], [PAULI_X], [3]) == cnotgate(basis(16), 7, 3)

Test Passed

In [66]:
bss = basis(16);
############# uncomment to run benchmarks ##################
@benchmark cnotgate($(bss), 7, 3)
#@benchmark general_controlled_gates(16, [P1], [7], [PAULI_X], [3])

BenchmarkTools.Trial: 
  memory estimate:  1.50 MiB
  allocs estimate:  8
  --------------
  minimum time:     127.956 μs (0.00% GC)
  median time:      752.830 μs (0.00% GC)
  mean time:        665.379 μs (15.52% GC)
  maximum time:     3.045 ms (73.27% GC)
  --------------
  samples:          7486
  evals/sample:     1

In [26]:
toffoligate(basis::Vector{DInt}, b1::Int, b2::Int, b3::Int) = multicontrolled_U1(basis, PAULI_X, [b1, b2], b3)

toffoligate (generic function with 2 methods)

In [27]:
# Assigned: LW
# multi-control gate for single qubit gate
function multicontrolled_U1(basis::Vector{DInt}, gate::PermuteMultiply, cbits::Vector{Int}, b2::Int)
end
# repeat for other matrix formats ...

multicontrolled_U1 (generic function with 1 method)

In [28]:
# TODO
# future, arbituary two site gate support
# probabily focus is enough? or, use projection decomposition?

In [29]:
#@test toffligate(3, 2, 3, 1) == TOFFOLI

In [30]:
x = zeros(Complex64, 0)
push!(x, 1.0)

1-element Array{Complex{Float32},1}:
 1.0+0.0im

In [31]:
takebit(12, 2) 

0

In [95]:
v = Diagonal([0, 1,2,3])
v[2,3]

0

In [110]:
size(basis(4),1)

16